# PROBLEMA 2: **Claificador de Sapm**


### Entrega: Daniel Rojo Mata

## **Enunciado del problema**

Descarga el conjunto de datos de spam disponible en [este enlace](http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/spam.csv) y realiza lo siguiente:

1. **Exploración de datos:**  
   - Reporta el porcentaje de correos etiquetados como *spam* y *no spam* en el conjunto de datos.

2. **División del conjunto de datos:**  
   - Divide aleatoriamente el conjunto de datos en:
     - **60 %** para entrenamiento  
     - **20 %** para validación  
     - **20 %** para prueba  
   - Usa `0` como semilla para el generador de números aleatorios.

3. **Entrenamiento de clasificadores:**  
   - Entrena **dos clasificadores Naive Bayes** con diferentes distribuciones.

4. **Evaluación de modelos:**  
   - Usa los clasificadores entrenados para predecir *spam* en los datos de **entrenamiento** y **validación**.  
   - Reporta el **porcentaje de predicciones correctas** de cada clasificador.

5. **Análisis del desempeño:**  
   - Discute el rendimiento de los diferentes clasificadores con base en los resultados obtenidos.

6. **Evaluación en el conjunto de prueba:**  
   - Reporta el **porcentaje de predicciones correctas** en el subconjunto de **prueba** utilizando el clasificador con mejor rendimiento en la validación.

## **Descripción del conjunto de datos**
El archivo `spam.csv` contiene **2001 valores por renglón**, distribuidos de la siguiente manera:
- Los **primeros 2000 valores** representan el **histograma de palabras** en un correo.
- El **último valor (posición 2000)** indica la clase del correo:
  - `1` → Es **spam**  
  - `0` → No es **spam**  


# **SOLUCIÓN**

### Librerías

In [ ]:
#Se importan los cosos necesarios 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB

## Clasificador con distribución Bernoulli

In [ ]:
class NaiveBayesBernoulli:
    def __init__(self):
        """
        Inicializa los atributos del modelo de Naive Bayes con distribución Bernoulli.
        """
        self.num_clases = None  # Número de clases en el conjunto de datos
        self.num_caracteristicas = None  # Número de características en los datos
        self.proba_caracteristica_dado_clase = None  # Matriz de probabilidades condicionales P(característica|clase)
        self.proba_clase = None  # Vector de probabilidades a priori P(clase)

    def fit(self, X_ent, y_ent):
        """
        Entrena el modelo con el conjunto de entrenamiento.

        Parámetros:
            X_ent: Matriz de características del conjunto de entrenamiento
            y_ent: Vector de etiquetas de clase del conjunto de entrenamiento
        """
        clases_unicas = np.unique(y_ent)  # Encuentra las clases únicas en el conjunto de entrenamiento
        self.num_clases = clases_unicas.size  # Almacena el número total de clases
        self.num_caracteristicas = X_ent.shape[-1]  # Obtiene el número de características por muestra
        num_muestras = X_ent.shape[0]  # Obtiene el número total de muestras en el conjunto de entrenamiento

        # Inicializa la matriz de probabilidades condicionales con ceros
        self.proba_caracteristica_dado_clase = np.zeros((self.num_clases, self.num_caracteristicas))  
        # Inicializa el vector de probabilidades a priori con ceros
        self.proba_clase = np.zeros(self.num_clases)  

        for i, clase in enumerate(clases_unicas):  # Itera sobre cada clase
            X_clase = X_ent[y_ent == clase]  # Filtra las muestras pertenecientes a la clase actual
            cuenta_atributos = np.count_nonzero(X_clase, axis=0)  # Cuenta cuántas veces cada atributo es 1 dentro de la clase
                                                                  # axis = 0 indica que se aplica a lo largo de las columnas
            num_muestras_clase = X_clase.shape[0]  # Obtiene el número de muestras en la clase actual
            
            # Calcula P(atributo=1 | clase) dividiendo la cuenta entre el número de muestras de la clase
            self.proba_caracteristica_dado_clase[i, :] = cuenta_atributos / num_muestras_clase # [i, :] := modifica toda la fila 'i' de la matriz 
            
            # Calcula P(clase) como la proporción de muestras de la clase en el total de datos
            self.proba_clase[i] = num_muestras_clase / num_muestras  
    
    def calcular_probabilidades_posteriores(self, X):
        """
        Calcula las probabilidades posteriores (proporcionales) para cada muestra:
            P(clase|datos) ∝ P(datos|clase) * P(clase)
        Se omite el denominador P(datos) ya que es constante para todas las clases.

        Parámetros: 
            X: Matriz de características del conjunto de prueba
        
        return: 
            Matriz de probabilidades de cada clase para cada muestra
        """
        probabilidades = np.zeros((X.shape[0], self.num_clases))  # Matriz para almacenar las probabilidades calculadas
                                                                  # Es de tamaño: NumCaracterísticas x NumClases
        
        for indice_clase in range(self.num_clases):  # Itera sobre cada clase
            # Calcula la verosimilitud P(X|C) usando la distribución de Bernoulli.
                # Bajo el supuesto de independencia condicional (Bayes ingenuo), se calcula como:
                # P(X|C) = P(x_1|C) * P(x_2|C) * P(x_3|C) * ... * P(x_n|C)
            verosimilitud = np.prod(self.bernoulli(X, self.proba_caracteristica_dado_clase[indice_clase, :]), axis=1)  
            # Multiplica la verosimilitud por la probabilidad a priori de la clase P(C)
            # Esto nos da una cantidad proporcional a la probabilidad posterior P(C|X)
            probabilidades[:, indice_clase] = verosimilitud * self.proba_clase[indice_clase]

        return probabilidades  # Retorna la matriz de probabilidades posteriores

    def predict(self, X):
        """
        Predice las clases de un conjunto de datos.
        
        Parámetros:
            X: Matriz de características del conjunto de prueba
        
        return: 
            Vector con las clases predichas
        """
        return np.argmax(self.calcular_probabilidades_posteriores(X), axis=1)  # Retorna la clase con la mayor probabilidad posterior
    
    @staticmethod
    def bernoulli(x, q):
        """
        Función de masa de probabilidad de la distribución de Bernoulli.
        
        Parámetros:
            x: Valores de la variable aleatoria (0 o 1)
            q: Probabilidad del éxito (atributo = 1) en la distribución de Bernoulli
        
        return: 
            Probabilidad de cada muestra bajo la distribución de Bernoulli
        """
        epsilon = 1e-9  # Pequeño valor para evitar cálculos con 0 o 1 exactos que puedan generar errores numéricos
        q = np.clip(q, epsilon, 1 - epsilon)  # Restringe los valores de q al intervalo (epsilon, 1 - epsilon)
        return q**x * (1.0 - q)**(1.0 - x)  # Aplica la fórmula de la distribución de Bernoulli

## Clasificador con distribución Gaussiana

Se utiliza el logaritmo para un mejor trabajo, pues multiplicar probabilidades pequeñas puede dar valores muy bajos, cercanos a cero, lo que puede causar subdesbordamiento numérico en la computadora.

El objetivo de usar logaritmo es evitar esto último.

In [ ]:
class NaiveBayesGaussiano:
    def __init__(self):
        """
        Inicializa los atributos del modelo de Naive Bayes con distribución Gaussiana.
        """
        self.media = None  # Media de cada característica por clase
        self.desviacion = None  # Desviación estándar de cada característica por clase
        self.proba_clase = None  # Probabilidad a priori de cada clase
        self.clases = None  # Clases únicas en el conjunto de datos
        self.epsilon = 1e-6  # Pequeño valor para evitar divisiones por 0 en la varianza

    def fit(self, X_ent, y_ent):
        """
        Entrena el modelo con el conjunto de entrenamiento.
        Calcula los parámetros de la distribución Gaussiana (media y desviación estándar) 
        para cada característica en cada clase.
        
        Parámetros:
            X_ent: ndarray de forma (num_muestras, num_características), matriz de datos de entrada.
            y_ent: ndarray de forma (num_muestras,), vector de etiquetas de clase.
        """
        self.clases = np.unique(y_ent)  # Obtiene las clases únicas en el conjunto de entrenamiento
        num_clases = len(self.clases)  # Número total de clases
        num_caracteristicas = X_ent.shape[1]  # Número de características en los datos
        
        # Inicializa matrices para almacenar la media, desviación estándar y probabilidades a priori, resp.
        self.media = np.zeros((num_clases, num_caracteristicas))
        self.desviacion = np.zeros((num_clases, num_caracteristicas))
        self.proba_clase = np.zeros(num_clases)
        
        # Itera sobre cada clase para calcular sus parámetros estadísticos
        for i, clase in enumerate(self.clases):
            X_clase = X_ent[y_ent == clase]  # Filtra las muestras que pertenecen a la clase actual
            self.media[i, :] = X_clase.mean(axis=0)  # Calcula la media de cada característica
            self.desviacion[i, :] = np.maximum(X_clase.std(axis=0), self.epsilon)  # Evita sigma = 0
            self.proba_clase[i] = len(X_clase) / len(X_ent)  # Calcula la probabilidad a priori de la clase

    def gaussian_log(self, x, mu, sigma):
        """
        Calcula la densidad de probabilidad de una distribución Gaussiana para cada característica.

        Parámetros:
            x: ndarray de forma (num_muestras, num_características), valores de entrada.
            mu: ndarray de forma (num_características,), media de la distribución Gaussiana.
            sigma: ndarray de forma (num_características,), desviación estándar de la distribución Gaussiana.

        return:
            Log-probabilidad de cada característica bajo la distribución Gaussiana correspondiente.
        """
        sigma = np.maximum(sigma, self.epsilon)  # Asegura que sigma no sea 0 para evitar errores numéricos
        coef = -0.5 * np.log(2 * np.pi * sigma**2)  # Cálculo del coeficiente de normalización en logaritmo
        exponente = -((x - mu) ** 2) / (2 * sigma ** 2)  # Cálculo de la parte exponencial de la función Gaussiana
        return coef + exponente  # Retorna la suma de log-probabilidades para estabilidad numérica

    def calcular_probabilidades_posteriores(self, X):
        """
        Calcula las probabilidades logarítmicas de cada clase para cada muestra.

        Parámetros:
            X: ndarray de forma (num_muestras, num_características), datos de entrada.

        return:
            log_probabilidades: ndarray de forma (num_muestras, num_clases),
                                matriz de log-probabilidades de cada muestra perteneciendo a cada clase.
        """
        num_muestras = X.shape[0]  # Número total de muestras
        num_clases = len(self.clases)  # Número total de clases
        log_probabilidades = np.zeros((num_muestras, num_clases))  # Inicializa matriz de log-probabilidades
        
        # Itera sobre cada clase para calcular la log-probabilidad de que una muestra pertenezca a ella
        for i in range(num_clases):
            log_prob_verosimilitud = self.gaussian_log(X, self.media[i, :], self.desviacion[i, :])  
            log_prob_verosimilitud = np.sum(log_prob_verosimilitud, axis=1)  # Suma las log-probabilidades de cada característica
            
            # Aplica la fórmula de la probabilidad posterior en logaritmo:
            # log P(C|X) = log P(X|C) + log P(C), sin dividir por P(X) ya que no afecta la clasificación
            log_probabilidades[:, i] = log_prob_verosimilitud + np.log(self.proba_clase[i]) 
        
        return log_probabilidades  # Retorna la matriz de log-probabilidades para cada muestra y clase

    def predict(self, X):
        """
        Predice la clase de cada muestra en X.

        Parámetros:
            X: ndarray de forma (num_muestras, num_características), datos de entrada.

        return:
            Un vector con las clases predichas para cada muestra.
        """
        log_probabilidades = self.calcular_probabilidades_posteriores(X)  # Calcula log-probabilidades para cada clase
        return self.clases[np.argmax(log_probabilidades, axis=1)]  # Retorna la clase con mayor log-probabilidad para cada muestra

## Procesamiento de la data

In [ ]:
archivo = "spam.csv"
data = pd.read_csv(archivo, 
                   header=None,
                   sep=None,
                   engine="python")

## Función para cálculo de precisión

In [ ]:
# Función para entrenar y evaluar los clasificadores
def entrenar_y_evaluar(clasificador, X_ent, y_ent, X_val, y_val, X_test, y_test):
    # Entrenar el clasificador
    clasificador.fit(X_ent, y_ent)
    
    # Predecir para los conjuntos de entrenamiento, validación y prueba
    predicciones_ent = clasificador.predict(X_ent)
    predicciones_val = clasificador.predict(X_val)
    predicciones_test = clasificador.predict(X_test)

    # Calcular la precisión (accuracy)
    accuracy_ent = accuracy_score(y_ent, predicciones_ent) * 100
    accuracy_val = accuracy_score(y_val, predicciones_val) * 100
    accuracy_test = accuracy_score(y_test, predicciones_test) * 100
    
    return accuracy_ent, accuracy_val, accuracy_test

## División del conjunto de datos

In [ ]:
# Separa las características y la etiqueta
X = data.iloc[:, :-1].to_numpy()  # Todas las columnas excepto la última
y = data.iloc[:, -1].to_numpy()   # La última columna (etiqueta)

# Dividir los datos en entrenamiento (60%), validación (20%) y prueba (20%)
X_ent, X_temp, y_ent, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

## Clasificadores Bernoulli y Gaussiano (incluidos los de sklearn)

In [ ]:
# Instancias de los clasificadores hechos a manita y con sklearn
clasificadores = {
    "Naive Bayes Bernoulli (sklearn)": BernoulliNB(),
    "Naive Bayes Gaussiano (sklearn)": GaussianNB(),
    "Naive Bayes Bernoulli (a manita)": NaiveBayesBernoulli(),
    "Naive Bayes Gaussiano (a manita)": NaiveBayesGaussiano(),
}

# Evaluar cada clasificador y mostrar resultados
for nombre, clasificador in clasificadores.items():
    print(f"Resultados con {nombre}:")
    accuracy_ent, accuracy_val, accuracy_test = entrenar_y_evaluar(clasificador, X_ent, y_ent, X_val, y_val, X_test, y_test)
    print(f"Precisión en entrenamiento: {accuracy_ent:.3f}%")
    print(f"Precisión en validación: {accuracy_val:.3f}%")
    print(f"Precisión en prueba: {accuracy_test:.3f}%")
    print("-" * 50)

## Reporte de Spam y No Spam

In [ ]:
spam_ratio = np.mean(y) * 100  # Asumiendo que spam = 1 y no spam = 0
print(f"Porcentaje de correos spam: {spam_ratio:.2f}%")
print(f"Porcentaje de correos no spam: {100 - spam_ratio:.2f}%")

## Mejor Clasificador

Se hace el comparativo de todos los clasificadores, incluídos los de sklearn.

In [ ]:
# Ahora, seleccionar el mejor clasificador con base en la precisión en validación
# Seleccionamos el clasificador que tuvo el mejor desempeño en validación
mejor_clasificador = max(clasificadores.items(),
                         key=lambda x: entrenar_y_evaluar(x[1],
                                                          X_ent,
                                                          y_ent,
                                                          X_val,
                                                          y_val,
                                                          X_test,
                                                          y_test)[1])

print(f"El clasificador con mejor rendimiento en el conjunto de validación es: {mejor_clasificador[0]}")
# Evaluamos su desempeño en el conjunto de prueba
accuracy_mejor_test = entrenar_y_evaluar(mejor_clasificador[1], X_ent, y_ent, X_val, y_val, X_test, y_test)[2]
print(f"Precisión en prueba para el mejor clasificador: {accuracy_mejor_test:.3f}%")

# **DISCUSIÓN**

### Sobre el Desempeño de los Clasificadores

1. **Mejor Desempeño del Clasificador Gaussiano**:
El clasificador **Naive Bayes Gaussiano** es el que presenta el mejor desempeño en validación (**93.04%**) y prueba (**92.46%**). Esto sugiere que este modelo es el más efectivo para los datos, posiblemente debido a que las características siguen una distribución más cercana a la normal, que es una suposición clave del modelo.

2. **Poca Variabilidad Entre los Modelos**:
Los resultados muestran que las variantes de Naive Bayes (Bernoulli y Gaussiano, tanto de `sklearn` como "a manita") tienen un desempeño similar, con diferencias mínimas. Esto indica que la implementación "a manita" está funcionando correctamente y produce resultados comparables a la versión de `sklearn`.

3. **Posible Sobreajuste del Modelo Gaussiano**:
El modelo Gaussiano presenta una precisión ligeramente superior en el conjunto de entrenamiento (**93.81%**) en comparación con validación (**93.04%**) y prueba (**92.46%**). Esto podría ser un indicio de sobreajuste, aunque la diferencia no es significativa, lo que sugiere que la generalización sigue siendo adecuada.

4. **Consistencia Entre Modelos**:
Los modelos **Bernoulli** tienen un desempeño ligeramente inferior (alrededor del **91%** en validación), indicando que este tipo de modelo puede ser menos adecuado para este conjunto de datos.

5. **Comparación de Precisión**:
Aunque todos los modelos muestran buen desempeño, el **Naive Bayes Gaussiano** tiene una ligera ventaja en precisión, lo que podría ser relevante para minimizar los falsos positivos o negativos.

# **CONCLUSIÓN**

El Naive Bayes Gaussiano es el modelo con mejor desempeño, alcanzando 93.04% en validación y 92.46% en prueba, lo que sugiere que se ajusta bien a los datos.

Las diferencias entre los modelos (Gaussiano y Bernoulli) son mínimas, lo que confirma que la implementación "a manita" (usando numpy) es correcta.

En general, todos los modelos ofrecen un buen desempeño, pero el Naive Bayes Gaussiano es la mejor opción para este problema.